# qHTS for Inhibitors of human tyrosyl-DNA phosphodiesterase 1 (TDP1): qHTS in cells in absence of CPT

## Introduction
Human tyrosyl-DNA phosphodiesterase 1 (TDP1) is a novel repair gene, and we propose to use it as a new target for anticancer drug development. TDP1 is not an essential protein, but under treatment with topoisomerase I poison (camptothecin: CPT), TDP1 works as a critical factor for cell survival. To directly identify novel TDP1 inhibitors active in a cellular environment, we have knocked-out the Tdp1 gene in chicken DT40 cells (Tdp1-/-) and generated a complemented counterpart cells that contains a stable transfection of the human TDP1 gene (Tdp1-/-;hTDP1 cells). For the primary screen, Tdp1-/-;hTDP1 cells will be exposed to small molecules in the presence or absence of CPT, and their growth kinetics will be evaluated after 48 hours by measuring ATP activity. If a given compound shows a synergistic effect with CPT, this compound could inhibit the repair pathway of CPT-induced lesions including the TDP1-mediated repair pathway. The hit compounds will then be evaluated in the presence or absence of CPT using Tdp1-/- cells. If a compound shows synergistic effect with CPT in Tdp1-/-;hTDP1 cells, but not with Tdp1-/- cells, such compound could be involved in the TDP1-mediated repair pathway inhibition. In tertiary assays, biochemical gel-based assays will be used to assess whether the hit compounds specifically target TDP1.

## Imports

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
from standardizer.CustomStandardizer import CustomStandardizer
from loaders.Loaders import CSVLoader
import sys
%matplotlib inline
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

sys.path.append('src')

## Initial exploration

### Carregar o dataset
The first step, analysing this dataset, includes loading and displaying TDP1 data.

In [ ]:
file = '../dataset/TDP1_activity_dataset.csv'
dataset = pd.read_csv(file, sep=',')
dataset.head()

### Simple Analyses
This following step was taken to analyse how data presents itself along the lines and collumns of the Datasets

In [ ]:
dataset.size

In [ ]:
dataset.shape

O nosso dataset foi carregado sobre o nome "dataset" que possui 40.000 moléculas distintas e 48 características da sua análise, totalizando em 1.920.000 valores.

In [ ]:
dataset.columns

| ColumnsName | Description |
| :-: | :-: |
| **PUBCHEM_RESULT_TAG** | This column contains an increasing number starting from one. |
| **PUBCHEM_SID** | PubChem SubstanceID |
| **PUBCHEM_CID** | PubChem CompoundID |
| **PUBCHEM_ACTIVITY_OUTCOME** | This field allows the submitter to make an expert judgment call about the activity of each test result. Using a number, the value is set to 1 (inactive) or 2 (active) based on whatever means appropriate. In addition to active/inactive, this field can also be set to 3 (inconclusive), 4 (unspecified) or 5 (probe). The 'probe' designation indicates that the activity of the test result has been tested and confirmed though multiple rounds of experimental inquiry |
| **PUBCHEM_ACTIVITY_SCORE** | The activity of a test result may be assigned a normalized score between 0 and 100 where the most active result rows have scores closer to 100 and inactive closer to 0, so that one can rank the result based on this data and prioritize hits |
| **PUBCHEM_ACTIVITY_URL** | An URL may optionally be provided for Assay Data reported for this Substance in this column. |
| **PUBCHEM_ASSAYDATA_COMMENT** | Textual annotation and comments |
| **Potency** | Concentration at which compound exhibits half-maximal efficacy |
| **Efficacy** | Maximal efficacy of compound, reported as a percentage of control |
| **Analysis Comment** | Annotation/notes on a particular compound's data or its analysis |
| **Activity_Score** | Activity score |
| **Curve_Description** | A description of dose-response curve quality |
| **Fit_LogAC50** | The logarithm of the AC50 from a fit of the data to the Hill equation (calculated based on Molar Units) |
| **Fit_HillSlope** | The Hill slope from a fit of the data to the Hill equation |
| **Fit_R2** | R^2 fit value of the curve. Closer to 1.0 equates to better Hill equation fit |
| **Fit_InfiniteActivity** | The asymptotic efficacy from a fit of the data to the Hill equation |
| **Fit_ZeroActivity** | Efficacy at zero concentration of compound from a fit of the data to the Hill equation |
| **Fit_CurveClass** | Numerical encoding of curve description for the fitted Hill equation |
| **Excluded_Points** | Which dose-response titration points were excluded from analysis based on outlier analysis |
| **Max_Response** | Maximum activity observed for compound (usually at highest concentration tested) |
| **Activity at xx uM*** | % Activity at given concentration |
| **Compound QC** | NCGC designation for data stage: 'qHTS', 'qHTS Verification', 'Secondary Profiling' |
| **smiles** | SMILES (Simplified Molecular Input Line Entry System) is a chemical notation that allows a user to represent a chemical structure in a way that can be used by the computer. |

*Activity at xx uM corresponde as atividades em todas em concentraçoes presentes no dataset.

In [ ]:
dataset.dtypes

In [ ]:
sub_dataset = dataset[['Potency', 'Efficacy', 'Fit_LogAC50', 'Fit_HillSlope', 'Fit_R2',
       'Fit_InfiniteActivity', 'Fit_ZeroActivity', 'Activity at 0.0000295000 uM',
       'Activity at 0.0000590000 uM', 'Activity at 0.0001503265 uM',
       'Activity at 0.0002712146 uM', 'Activity at 0.0005895491 uM',
       'Activity at 0.00117 uM', 'Activity at 0.00179 uM',
       'Activity at 0.00299 uM', 'Activity at 0.00672 uM',
       'Activity at 0.014 uM', 'Activity at 0.026 uM', 'Activity at 0.040 uM',
       'Activity at 0.074 uM', 'Activity at 0.167 uM', 'Activity at 0.363 uM',
       'Activity at 0.628 uM', 'Activity at 0.975 uM', 'Activity at 1.849 uM',
       'Activity at 4.119 uM', 'Activity at 9.037 uM', 'Activity at 15.83 uM',
       'Activity at 21.08 uM', 'Activity at 46.23 uM', 'Activity at 92.54 uM',
       'Activity at 165.6 uM']]

sub_dataset.describe()

## Pre-Processing

De seguida, o número de valores nao atríbuidos serao contados tanto na totalidade como por coluna.

### Visualization of the NA's

In [ ]:
print(dataset.isna().sum())
print(f"TOTAL: {dataset.isna().sum().sum()}")

In [ ]:
msno.bar(dataset,  sort="ascending")

Podemos observar que existem algumas colunas completamente constituidas por NA's, como "PUBCHEM_ASSAYDATA_COMMENT" e "Analysis Comment". Desta forma, estas colunas nao fornecem qualquer tipo de informaçao do dataset.
Destaca-se a ausencia de smile em 10 das moléculas.

É possível observar que 50,3% dos valores sao NA's.

### Drop specific features

In [ ]:
dataset = dataset.dropna(axis=1, how='all')
dataset.drop(['PUBCHEM_ACTIVITY_URL', 'Compound QC'], axis=1)
dataset = dataset[dataset['smiles'].notna()]

print(dataset.shape)
print(dataset.columns)

Foram removidas 3 colunas constituidas apenas por NA's, o que permitiu reduzir o dataset para 45 colunas no total.
Também foram removidas colunas cuja informaçao nao será útil para as analises posteriores. Mais específicamente, foram removidas as colunas "PUBCHEM_ACTIVITY_URL" e "Compound QC", reduzindo assim o total de colunas para 43.
As 10 moléculas que não possuiam notação SMILE foram removidas do dataset.

### Graphic Analyses
#### Activity_outcome and Phenotype

In [ ]:
activity = dataset.groupby('PUBCHEM_ACTIVITY_OUTCOME').size()
labels_activity = dataset.groupby('PUBCHEM_ACTIVITY_OUTCOME').size().index
dataset.groupby('PUBCHEM_ACTIVITY_OUTCOME').size()

In [ ]:
fenotipo = dataset.groupby('Phenotype').size()
labels_fenotipo = dataset.groupby('Phenotype').size().index
dataset.groupby('Phenotype').size()

#### Pie Charts Activity_outcome and Phenotype
MUDAR PARA BARRAS

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.pie(activity, labels=labels_activity, autopct='%1.1f%%', startangle=90)
ax1.set_title('PUBCHEM_Activity_Outcome')
ax2.pie(fenotipo, labels=labels_fenotipo, autopct='%1.1f%%', startangle=360)
ax2.set_title('Phenotype')

#### Boxplots of Activity at 46.23 uM, 1.849 uM, 0.363 uM, 0.00299 uM and 9.037 uM
REVER

In [ ]:
plt.subplots(figsize=(10, 10))
sns.set(font_scale=1.4)
plt.title("Activity at 0.00299 uM", fontsize=25)
sns.boxplot(y="Activity at 0.00299 uM",
            data=dataset, palette="Set3")

In [ ]:
plt.subplots(figsize=(10, 10))
sns.set(font_scale=1.4)
plt.title("Activity at 0.363 uM", fontsize=25)
sns.boxplot(y="Activity at 0.363 uM",
            data=dataset, palette="Set3")

In [ ]:
plt.subplots(figsize=(10, 10))
sns.set(font_scale=1.4)
plt.title("Activity at 1.849 uM", fontsize=25)
sns.boxplot(y="Activity at 1.849 uM",
            data=dataset, palette="Set3")

In [ ]:
plt.subplots(figsize=(10, 10))
sns.set(font_scale=1.4)
plt.title("Activity at 9.037 uM", fontsize=25)
sns.boxplot(y="Activity at 9.037 uM",
            data=dataset, palette="Set3")

In [ ]:
plt.subplots(figsize=(10, 10))
sns.set(font_scale=1.4)
plt.title("Activity at 46.23 uM", fontsize=25)
sns.boxplot(y="Activity at 46.23 uM",
            data=dataset, palette="Set3")

## Standardize molecules

In [ ]:
def standardize(dataset, id_field ,mols_field,class_field):

    loader = CSVLoader(dataset,
                       id_field=id_field,
                       mols_field = mols_field,
                       labels_fields = class_field)

    dataset = loader.create_dataset()

    standardisation_params = {
        'REMOVE_ISOTOPE': True,
        'NEUTRALISE_CHARGE': True,
        'REMOVE_STEREO': False,
        'KEEP_BIGGEST': True,
        'ADD_HYDROGEN': False,
        'KEKULIZE': True,
        'NEUTRALISE_CHARGE_LATE': True}

    CustomStandardizer(params = standardisation_params).standardize(dataset)

    return dataset

In [ ]:
dataset = standardize(file, "PUBCHEM_CID", "smiles", "PUBCHEM_ACTIVITY_OUTCOME")
dataset.save_to_csv("../dataset/standardized.csv")

In [ ]:
dataset = pd.read_csv("../dataset/standardized.csv")

In [ ]:
# from rdkit.Chem import MolFromSmiles, Descriptors, AllChem, rdMolDescriptors
from rdkit.Chem import Descriptors, AllChem
from rdkit import Chem, DataStructs

def get_molecular_descriptors(molecules):
    descrip = np.zeros((len(molecules), 14))

    i = 0

    for molec in molecules:
        descrip[i, 0] = Descriptors.ExactMolWt(molec)
        descrip[i, 1] = Chem.Crippen.MolLogP(molec)
        descrip[i, 2] = Chem.Lipinski.RingCount(molec)
        descrip[i, 3] = Chem.Lipinski.NumAliphaticCarbocycles(molec)
        descrip[i, 4] = Chem.Lipinski.NumAliphaticHeterocycles(molec)
        descrip[i, 5] = Chem.Lipinski.NumAromaticRings(molec)
        descrip[i, 6] = Descriptors.TPSA(molec)
        descrip[i, 7] = Chem.Lipinski.NHOHCount(molec)
        descrip[i, 8] = Chem.Lipinski.NOCount(molec)
        descrip[i, 9] = Chem.Lipinski.NumHAcceptors(molec)
        descrip[i, 10] = Chem.Lipinski.NumHDonors(molec)
        descrip[i, 11] = Descriptors.NumValenceElectrons(molec)
        descrip[i, 12] = Chem.Lipinski.NumRotatableBonds(molec)
        descrip[i, 13] = Descriptors.NumRadicalElectrons(molec)
        i += 1

    columns = ["Molecular Weight", "LogP", "Ring Count", "NumAliphaticCarbocycles", "NumAliphaticHeterocycles",
               "NumAromaticRings", "TPSA", "NHOHCount", "NOCount", "NumHAcceptors", "NumHDonors", "NumValenceElectrons",
               "NumRotatableBonds", "NumRadicalElectrons"]

    df = pd.DataFrame(descrip, columns=columns)
    return df

In [ ]:
molecules = [Chem.MolFromSmiles(smile) for smile in dataset.mols]

In [ ]:
molecular_descriptors = get_molecular_descriptors(molecules)
molecular_descriptors["activity"] = dataset.y

In [ ]:
molecular_descriptors

In [ ]:
def generate_box_plot(feature, class_name, title, dataframe, orientation):
    plt.subplots(figsize=(20, 10))
    sns.set(font_scale=1.4)
    plt.title(title, fontsize=25)
    sns.boxplot(x=feature, y=class_name, orient=orientation,
                data=dataframe, palette="Set3")


def generate_multiple_box_plots(dataframe, title, columns, class_name):
    plt.subplots(figsize=(20, 10))
    sns.set(font_scale=1.4)
    plt.title(title, fontsize=25)
    columns.append(class_name)
    sns.boxplot(x="variable", y="value", data=pd.melt(dataframe.loc[:, columns], class_name), hue=class_name,
                palette="Set3")

In [ ]:
generate_box_plot("Molecular Weight", "activity", "", molecular_descriptors, "h")

In [ ]:
columns = ["Ring Count", "NumAromaticRings"]

generate_multiple_box_plots(molecular_descriptors, "", columns, "activity")

## Feature generation

In [ ]:
from compoundFeaturization.rdkitFingerprints import MorganFingerprint
from loaders.Loaders import CSVLoader

loader = CSVLoader("standardized.csv",
                   mols_field='mols',
                   labels_fields='y')

dataset = loader.create_dataset()

### Generate fingerprints

In [ ]:
MorganFingerprint().featurize(dataset)

### Generate molecular descriptors

In [ ]:
from compoundFeaturization.rdkitDescriptors import TwoDimensionDescriptors
from scalers.sklearnScalers import StandardScaler

scaler = StandardScaler()
TwoDimensionDescriptors().featurize(dataset, scaler=scaler)

## Feature Selection

In [ ]:
from featureSelection.baseFeatureSelector import LowVarianceFS, KbestFS, BorutaAlgorithm

In [ ]:
LowVarianceFS().select_features(dataset)

In [ ]:
dataset.y

In [ ]:
import numpy as np
np.any(np.isnan(dataset.X))

In [ ]:
BorutaAlgorithm(max_iter=10, n_estimators=100).select_features(dataset)

In [ ]:
dataset.X

## Unsupervised exploration

In [ ]:
from rdkit import Chem, DataStructs
from copy import copy

In [ ]:
import numpy as np
def generate_similarities(fps):
    similarities_list = []
    bv1 = DataStructs.ExplicitBitVect(fps.shape[1])

    new_fps = []
    for fp in fps:
        bv12 = copy(bv1)
        for i,bit in enumerate(fp):
            if bit == 1:
               bv12.SetBit(i)
        new_fps.append(bv12)

    for i in range(0,2000):
        remaining_fp = new_fps[:i] + new_fps[i+1:]
        similarities = DataStructs.BulkTanimotoSimilarity(new_fps[i], remaining_fp)
        similarities.insert(i,1)
        similarities_list.append(similarities)

    return np.transpose(np.array(similarities_list))


In [ ]:
similarities = generate_similarities(dataset.X)

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)

# Transform the data
df = pca.fit_transform(similarities)

tsne_df = TSNE(n_components=2).fit_transform(df)

In [ ]:
labels = dataset.y

plt.figure(figsize=(20, 20))
sns.scatterplot(
    df[:, 0], df[:, 1],
    hue=labels,
    palette=sns.color_palette("deep", 2),
    legend="full",
    s=25
)

In [ ]:
labels = dataset.y

plt.figure(figsize=(20, 20))
sns.scatterplot(
    tsne_df[:, 0], tsne_df[:, 1],
    hue=labels,
    palette=sns.color_palette("deep", 2),
    legend="full",
    s=25
)